# Read data from NVDB and convert to RDF for comparision with Overture Maps

Libraries and parameters

In [1]:
import sys, datetime
from rdflib import Graph, Namespace
import nvdbapiv3
from nvdb2rdf import *
import geopandas as gpd
from shapely import Point

# Parameters
rootPath = "C:\\Data\\GitHub\\jetgeo\\OM2ANY"

#NVDB Paths
nvdbOtlFilePath = "C:\\Data\\GitHub\\jetgeo\\OM2ANY\\OWL\\nvdb\\nvdb-owl.ttl"
nvdbVoIRI = "https://ontologi.atlas.vegvesen.no/nvdb/core/nvdb-owl/vegobjekt#"
nvdbVnIRI = "https://ontologi.atlas.vegvesen.no/nvdb/core/vegnett#"
nvdbOtlIRI = "https://ontologi.atlas.vegvesen.no/nvdb/core/nvdb-owl#"

#INSPIRE Paths
itnrOtlFilePath = "C:\\Data\\GitHub\\jetgeo\\OM2ANY\\OWL\\inspire\\itnr-owl.ttl"
itnrOtlIRI = "http://inspire.ec.europa.eu/ont"
featuretypeid = 105
knr = 3403


Føyer C:\DATA\GitHub\vegvesen\NVDB-Datakatalogen\owl til søkestien


Read ontologies

In [ ]:
# ---------------------------------------------------------------------------------------------
startTime = datetime.datetime.now()
# ---------------------------------------------------------------------------------------------
# Leser NVDB-ontologien til graf for objekttypebiblioteker
print(str(datetime.datetime.now()) + ' Leser inn NVDB-OTL fra ', nvdbOtlFilePath)
otl_nvdb = Graph()
otl_nvdb.parse(nvdbOtlFilePath, format="turtle")
# Leser INSPIRE-ontologien til egen graf
print(str(datetime.datetime.now()) + ' Leser inn INSPIRE-OTL fra ', itnrOtlFilePath)
otl_itnr = Graph()
otl_itnr.parse(itnrOtlFilePath, format="turtle")
#Slår sammen alle ontologier til en stor graf
print(str(datetime.datetime.now()) + ' Slår sammen ontologiene')
otl = Graph()
otl = otl_nvdb + otl_itnr 
# Setter opp graf og namespace-forkortelser for NVDB-data
print(str(datetime.datetime.now()) + ' Setter opp graf og namespace-forkortelser for data')
g_nvdb=Graph()
g_nvdb.bind("nvdb_vo", Namespace(nvdbVoIRI)) #IRI for NVDB-Objekter
g_nvdb.bind("nvdb_vn", Namespace(nvdbVnIRI)) #IRI for NVDB Vegnett
g_nvdb.bind("nvdb_otl",Namespace(nvdbOtlIRI))
g_nvdb.bind("gsp",'http://www.opengis.net/ont/geosparql#')
g_nvdb.bind("nvdb_otl",Namespace(nvdbOtlIRI))


# ---------------------------------------------------------------------------------------------

Read data from NVDB and convert to RDF

In [ ]:
g_nvdb = nvdb2graph(featuretypeid, knr, otl_nvdb)

Print to Turtle

In [ ]:
fileName = rootPath + "\\data\\nvdb_" + str(featuretypeid) + "_" + str(knr) + ".ttl."
print(str(datetime.datetime.now()) + ' Skriver til NVDB-Turtle-fil: ' + fileName)
g_nvdb.serialize(destination=fileName, format="turtle")

Read road network from NVDB and create GeoDataFrame with nodes

In [6]:
v = nvdbapiv3.nvdbVegnett()
v.filter( { 'kommune' : knr } )
print("Filter: " + str(v.filterdata))

# Create an empty list to store dictionaries
data_list = []	

ettNvdbObj = v.nesteForekomst()
while ettNvdbObj: 
	#print (ettNvdbObj)

	wkt_coordinates = ettNvdbObj['geometri']['wkt']
	startnode = ettNvdbObj['startnode']
	endnode = ettNvdbObj['sluttnode']
	
	# Extract the first and last coordinate triple from 'wkt'
	coordinates_list = wkt_coordinates.split(', ')
	first_coordinate = coordinates_list[0].replace('LINESTRING Z(', '').split()
	last_coordinate = coordinates_list[-1].replace(')', '').split()

	# print(f"Veglenkesegment: {ettNvdbObj['veglenkesekvensid']} {ettNvdbObj['referanse']}" )
	# print(f"Startnode: {startnode} (First Coordinate: {first_coordinate})")
	# print(f"Endnode: {endnode} (Last Coordinate: {last_coordinate})")
	# print("-" * 50)

    # Parse coordinates into Point objects
	start_point = Point(float(first_coordinate[0]), float(first_coordinate[1]))
	end_point = Point(float(last_coordinate[0]), float(last_coordinate[1]))
    # Create dictionaries for start and end nodes and append to the list
	dict = {'node_id': ettNvdbObj['startnode'], 'geometry': start_point}
	data_list.append(dict)	
	dict = {'node_id': ettNvdbObj['sluttnode'], 'geometry': end_point}
	data_list.append(dict)
	
	ettNvdbObj = v.nesteForekomst()

# Create a GeoDataFrame from the list of dictionaries
gdf = gpd.GeoDataFrame(data_list)
gdf.drop_duplicates(subset='node_id', inplace=True)
# Print the GeoDataFrame
print(gdf)	

# Export the GeoDataFrame to GeoJSON format
gdf.to_file(rootPath + "\\data\\NVDB\\result.geojson", driver="GeoJSON")

# Print a success message
print("GeoDataFrame exported to result.geojson")


Filter: {'kommune': 3403}
Veglenkesegment: 650067 650067-1-3
Startnode: 725721 (First Coordinate: ['297063.4', '6749772.09', '250.619'])
Endnode: 3046118 (Last Coordinate: ['297075.8', '6749787.09', '250.519'])
--------------------------------------------------
Veglenkesegment: 650067 650067-2-3
Startnode: 3046118 (First Coordinate: ['297075.8', '6749787.09', '250.519'])
Endnode: 2265258 (Last Coordinate: ['297390.103', '6751979.667', '291.262'])
--------------------------------------------------
Veglenkesegment: 650067 650067-3-2
Startnode: 725871 (First Coordinate: ['297388.51', '6752007.34', '292.21'])
Endnode: 1775363 (Last Coordinate: ['297384.69', '6752094.75', '294.42'])
--------------------------------------------------
Veglenkesegment: 650067 650067-4-2
Startnode: 1775363 (First Coordinate: ['297384.69', '6752094.75', '294.42'])
Endnode: 1476076 (Last Coordinate: ['297382.545', '6752122.946', '294.973'])
--------------------------------------------------
Veglenkesegment: 65006